In [2]:
# 베르누이 나이브베이즈 모형은 둘 중에 조건부확률이 무엇이 더 큰지 알아보는 모델
# 스무딩은 한 세트의 가짜 데이터를 생성한다

### 다항분포 나이브베이즈 모형

다항분포 나이브베이즈 모형 클래스 `MultinomialNB`는 가능도 추정과 관련하여 다음 속성을 가진다.

* `feature_count_`: 각 클래스 $k$에서 $d$번째 면이 나온 횟수 $N_{d,k}$
* `feature_log_prob_`: 다항분포의 모수의 로그
 
$$ \log \mu_k = (\log \mu_{1,k}, \ldots, \log \mu_{D, k}) = \left( \log \dfrac{N_{1,k}}{N_k}, \ldots, \log \dfrac{N_{D,k}}{N_k} \right)$$
 
여기에서 $N_k$은 클래스 $k$에 대해 주사위를 던진 총 횟수를 뜻한다.

스무딩 공식은

$$ \hat{\mu}_{d,k} = \frac{ N_{d,k} + \alpha}{N_k + D \alpha} $$ 

이다.

이번에도 스팸 메일 필터링을 예로 들어보다. 다만 BOW 인코딩을 할 때, 각 키워드가 출현한 빈도를 직접 입력 변수로 사용한다.

In [30]:
X = np.array([
    [3, 4, 1, 2],
    [3, 5, 1, 1],
    [3, 3, 0, 4],
    [3, 4, 1, 2],
    [1, 2, 1, 4],
    [0, 0, 5, 3],
    [1, 2, 4, 1],
    [1, 1, 4, 2],
    [0, 1, 2, 5],
    [2, 1, 2, 3]])
y = np.array([0, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [31]:
from sklearn.naive_bayes import MultinomialNB
model_mult = MultinomialNB().fit(X, y)

사전 확률은 다음과 같이 구한다.

In [32]:
model_mult.classes_

array([0, 1])

In [33]:
model_mult.class_count_

array([4., 6.])

In [34]:
np.exp(model_mult.class_log_prior_)

array([0.4, 0.6])

다음으로 각 클래스에 대한 가능도 확률분포를 구한다. 다항분포 모형을 사용하므로 각 클래스틑 4개의 면을 가진 주사위로 생각할 수 있다. 그리고 각 면이 나올 확률은 각 면이 나온 횟수를 주사위를 던진 전체 횟수로 나누면 된다. 우선 각 클래스 별로 각각의 면이 나온 횟수는 다음과 같다. 

In [35]:
fc = model_mult.feature_count_
fc

array([[12., 16.,  3.,  9.],
       [ 5.,  7., 18., 18.]])

이 데이터에서 클래스 Y=0인 주사위를 던진 횟수는 첫번째 행의 값의 합인 40이므로 클래스 Y=0인 주사위를 던져 1이라는 면이 나올 확률은 다음처럼 계산할 수 있다.

$$ \mu_{1,Y=0} = \dfrac{12}{40} = 0.3 $$

In [36]:
fc / np.repeat(fc.sum(axis=1)[:, np.newaxis], 4, axis=1)

array([[0.3       , 0.4       , 0.075     , 0.225     ],
       [0.10416667, 0.14583333, 0.375     , 0.375     ]])

실제로는 극단적인 추정을 피하기 위해 이 값을 가중치 1인 스무딩을 한 추정값을 사용한다.

In [37]:
model_mult.alpha

1.0

In [38]:
(fc + model_mult.alpha) / \
    (np.repeat(fc.sum(axis=1)[:, np.newaxis],
               4, axis=1) + model_mult.alpha * X.shape[1])

array([[0.29545455, 0.38636364, 0.09090909, 0.22727273],
       [0.11538462, 0.15384615, 0.36538462, 0.36538462]])

이렇게 구한 모수 추정치는 다음과 같다.

In [39]:
theta = np.exp(model_mult.feature_log_prob_)
theta

array([[0.29545455, 0.38636364, 0.09090909, 0.22727273],
       [0.11538462, 0.15384615, 0.36538462, 0.36538462]])

이제 이 값을 사용하여 예측을 해 보자. 만약 어떤 메일에 1번부터 4번까지의 키워드가 각각 10번씩 나왔다면 다음처럼 확률을 구할 수 있다. 구해진 확률로부터 이 메일이 스팸임을 알 수 있다. 

In [40]:
x_new = np.array([10, 10, 10, 10])
model_mult.predict_proba([x_new])

array([[0.38848858, 0.61151142]])

다항분포의 확률질량함수을 사용하면 다음처럼 직접 확률을 구할 수도 있다.

In [41]:
p = (theta ** x_new).prod(axis=1)*np.exp(model_bern.class_log_prior_)
p / p.sum()

array([0.38848858, 0.61151142])

#### 연습 문제 3

MNIST 숫자 분류문제를 다항분포 나이브베이즈 모형을 사용하여 풀고 이진화(Binarizing)를 하여 베르누이 나이브베이즈 모형을 적용했을 경우와 성능을 비교하라.


#### 연습 문제 4

텍스트 분석에서 TF-IDF 인코딩을 하면 단어의 빈도수가 정수가 아닌 실수값이 된다. 이런 경우에도  다항분포 모형을 적용할 수 있는가?


In [1]:
# 현실적으로 텍스트 분석에서 쓸 수 있는 분석 모델은 거의 나이브베이즈밖에 없다. 현실적으로 계산량이 되는 것이.
# TF-IDF 인코딩은 이 단어가 들어가 있는 문서의 수 만큼 나눠주는 형태 말한다. 가중치를 주게 되면 정수가 아니게 된다.
    # 다항분포 모델 적용할 수 있나? 다항분포는 정수를 가정한 모델인데 그래도 잘 된다
    # 독립변수 카운트 할 때의 실제 코드는 sum으로 돼 있고 3번 나왔거나 3.1번 나왔거나 비슷한 값이 되기 떄문
    # 실제로 꼭 정수가 아니어도 된다
    # 이는 베르누이 나이브베이즈에도 적용된다.
    # 이론 상으로는 가우시안은 실수, 베르누이는 0,1, 다항분포는 정수를 받아야 하지만 실제로는 뭐든 간에 어디에 집어넣어도 동작한다
    # 물론 가정이랑 잘 맞지 않으면 성능이 좋지 않지만 에러가 나진 않는다
    # 그래서 TF_IDF 인코딩 해도 상관 없다

### 뉴스그룹 분류

다음은 뉴스그룹 데이터에 대해 나이브베이즈 분류모형을 적용한 결과이다. 

In [12]:
from sklearn.datasets import fetch_20newsgroups

news = fetch_20newsgroups(subset="all")
X = news.data
y = news.target

from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model1 = Pipeline([
    ('vect', CountVectorizer()),
    ('model', MultinomialNB()),
]) # 백터라이즈 + 다항분포 나이브베이즈
model2 = Pipeline([
    ('vect', TfidfVectorizer()),
    ('model', MultinomialNB()),
]) # Tfid 벡터라이즈 + 다항분포 나이브베이즈
model3 = Pipeline([
    ('vect', TfidfVectorizer(stop_words="english")),
    ('model', MultinomialNB()),
]) # Tfid 벡터라이즈 + 불용어 + 다항분포 나이브베이즈
model4 = Pipeline([
    ('vect', TfidfVectorizer(stop_words="english",
                             token_pattern=r"\b[a-z0-9_\-\.]+[a-z][a-z0-9_\-\.]+\b")),
    ('model', MultinomialNB()),
]) # Tfid 벡터라이즈 + 불용어 + 정규표현식 활용한 불필요한 토큰 제거 + 다항분포 나이브베이즈
# 점차적인 성능 개선

In [10]:
# 파이프라인: 예측 모델을 만들 때 안에서 이뤄지는 프리프로세싱, fit(), transform() 등을 겉에서 보면 마치 하나의 과정인 것처럼 보이게 해준다
# 겉으로는 fit()과 predict() 명령어만 써주면 된다
# 'vect', 'model'은 이름 바꿔줘도 됨

In [43]:
%%time
from sklearn.model_selection import cross_val_score, KFold

for i, model in enumerate([model1, model2, model3, model4]):
    scores = cross_val_score(model, X, y, cv=5)
    print(("Model{0:d}: Mean score: {1:.3f}").format(i + 1, np.mean(scores)))

Model1: Mean score: 0.855
Model2: Mean score: 0.856
Model3: Mean score: 0.883
Model4: Mean score: 0.888
CPU times: user 2min 15s, sys: 3.48 s, total: 2min 18s
Wall time: 1min 56s


In [15]:
# Tfid 성능 개선 미미
# 불용어, 정규표현식 등 활용하면 성능 크게 개선.
# 나이브 베이즈 모델에서는 쓸데없는 것을 빼는 것이 중요
# 나이브베이즈모델은 분류 중에서도 특히 텍스트 분류에 많이 쓰게 된다!

### 일반적인 실수값은 가우시안(정규분포) 나이브베이즈 모델, 0과 1이 기본이면 베르누이 나이브베이즈 모델, 1, 2,3  등으로 분포 나올 때는 다항분포 나이브베이즈 모델 쓰는 게 정상이다
### 세 경우 모두 섞여 있으면 각기 구간 나눠서 구한다

#### 연습 문제 5

(1) 만약 독립변수로 실수 변수, 0 또는 1 값을 가지는 변수, 자연수 값을 가지는 변수가 섞여있다면 사이킷런에서 제공하는 나이브베이즈 클래스를 사용하여 풀 수 있는가?

(2) 사이킷런에서 제공하는 분류문제 예제 중 숲의 수종을 예측하는 covtype 분류문제는 연속확률분포 특징과 베르누이확률분포 특징이 섞여있다. 이 문제를 사이킷런에서 제공하는 나이브베이즈 클래스를 사용하여 풀어라.

In [16]:
# covtype 분류 문제의 경우 앞 부분에는 가우시안 분포를 이루고 뒤에는 베르누이 분포를 이루고 있음
# 요지는 P(y|x)를 구하면 된다는 것